Extracción Transformación y Carga de los Datos 

Paso 1: Cargamos al DataFrame

In [217]:
import pandas as pd   # Libreria Necesaria para manipular los datos y estructura.
import ast            # Libreria necesario para la desanidacion de campos en varias columnas.

# Cargamos el Data Frame con el origen o fuente de los datos (archivo CSV)
df = pd.read_csv('Dataset/Originales/movies_dataset.csv')

C:\Users\Dalgy\AppData\Local\Temp\ipykernel_19408\2671923904.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Dataset/Originales/movies_dataset.csv')


Paso 2: Tratamiento de los datos

In [218]:
df = df.drop(columns=['adult','homepage','poster_path', 'imdb_id', 'original_title','video'])
# Se eliminaron estas columnas segun la guia, no seran necesarias en esta oportunidad.

In [219]:
# Validamos de nuevo el total de nulos por campo de la data
df.isnull().sum().sort_values(ascending=False)

belongs_to_collection    40972
tagline                  25054
overview                   954
runtime                    263
status                      87
release_date                87
original_language           11
vote_average                 6
revenue                      6
spoken_languages             6
vote_count                   6
title                        6
popularity                   5
production_countries         3
production_companies         3
budget                       0
id                           0
genres                       0
dtype: int64

In [220]:
# Segun el listado anterior de campos y su total de nulos por cada campo, iniciamos transformando los datos nulos listados en el EDA

df['belongs_to_collection'].isnull().sum() ## Contamos los nulos
# Verificamos y encontramos para la columna 'belongs_to_collection' 40.972 valores en nulo.
# El valor de los no nulos es cadena que contiene un formato diccionario, por lo tanto rellenamos los nulos con una lista de diccionario vacio.
df['belongs_to_collection'] = df['belongs_to_collection'].fillna('{}')

In [221]:
df['tagline'].value_counts(dropna=False).sort_values(ascending=False)
# Verificamos y encontramos para la columna 'tagline' 25.054 valores en nulo.
# El valor de los no nulos es cadena sin algun tipo de formato, por lo tanto rellenamos los nulos con la opcion para el modelo "sin preferencias".
df['tagline'] = df['tagline'].fillna('sin preferencias')

In [222]:
df['overview'].value_counts(dropna=False).sort_values(ascending=False)
# Verificamos y encontramos para la columna 'overview' 954 valores en nulo.
# Tambien encontramos redundancia entre el valor "No overview found" y "No Overview", por lo que reemplazaremos con "sin descripcion".
# Rellenaremos los campos NaN de la misma forma que el paso anterior "sin descripcion", para unificar valores
df['overview'] = df['overview'].replace(['No overview found.', 'No Overview'], 'Sin descripción')
# Rellenar NaNs
df['overview'] = df['overview'].fillna('Sin descripción')

Runtime

In [223]:
# Verificamos de nuevo y encontramos para la columna 'runtime' 
# Esta columna contiene valores numericos por lo tanto realizaremos varias consultas.
# 1 Contar los NaN y valores en cero
numero_nan_total = df['runtime'].isnull().sum()
# Mostrar la cantidad de NaN
print(f"Número total de NaN : {numero_nan_total}")
conteo_ceros = (df['runtime'] == 0).sum()
# Mostrar la cantidad de ceros
print(f"Número de valores en cero: {conteo_ceros}")

Número total de NaN : 263
Número de valores en cero: 1558


In [224]:
# 2 Consultamos algunos valore altos y bajos para identificar outliers 
valores_mas_altos = df['runtime'].nlargest(5)
# Mostrar los resultados
print("10 valores más altos:")
print(valores_mas_altos)

# Listar los 10 valores más bajos
valores_mas_bajos = df['runtime'].nsmallest(5)
# Mostrar los resultados
print("10 valores más bajos:")
#print(valores_mas_bajos)

# Analisis: los datos presentan uniformidad y no se evidencia una distancia atipica.

10 valores más altos:
24178    1256.0
19965    1140.0
40938    1140.0
13767     931.0
13953     925.0
Name: runtime, dtype: float64
10 valores más bajos:


In [225]:
# 3 Consultar por rangos para analisar la distribucion de los datos en "runtime"

# Definir los límites de los bins
bins = range(0, 1400, 100)  # Desde 0 hasta 1300, cada 100
# Crear las etiquetas para cada intervalo
labels = [f'{i}-{i + 99}' for i in bins[:-1]]
# Usar pd.cut() para categorizar 'runtime'
df['intervalo'] = pd.cut(df['runtime'], bins=bins, right=False, labels=labels)
# Contar cuántos valores hay en cada intervalo
conteo_intervalos = df['intervalo'].value_counts().sort_index()
print(conteo_intervalos)
# Mostrar los resultados
# print(conteo_intervalos)
# Analisis, fragmentacion de los datos nos muestra que la mayoria de datos estan entre valores de 0 a 100, seguido de 100 a 200, 
# entre esos dos rangos esta la mayor cantidad de datos.

# Eliminar la solo la columna 'intervalo', pues la usamos para entender la distribucion de los datos.
# df = df.drop('intervalo', axis=1)
df = df.drop(columns=['intervalo'])

intervalo
0-99         27774
100-199      17077
200-299        232
300-399         65
400-499         23
500-599         14
600-699          5
700-799          3
800-899          4
900-999          3
1000-1099        0
1100-1199        2
1200-1299        1
Name: count, dtype: int64


In [226]:
# Procedimiento para la columna "runtime"
# Los 263 datos NaN, seran transformados en el grupo de los ceros de 1558
df['runtime'] = df['runtime'].fillna(0)

In [227]:
df['status'].value_counts(dropna=False).sort_values(ascending=False)
# Encontramos 87 datos NaN, entre los demas estados del registro.
# Procedemos a rellenar con un estado denominado "sin clasificar"
df['status'] = df['status'].fillna('unclassified')

In [228]:
'''Requerimiento del Proyecto'''

# Por ser un requerimiento del proyecto se procede a eliminar las fechas nulas
df['release_date'].isnull().sum()  # contamos los nulos
# Eliminar filas donde la columna 'fecha' sea nula (NaN)
# Encontramos 87 valores nulos en esta columna.
df = df.dropna(subset=['release_date'])

#Nota: Es un campo muy importante en el proyecto, por lo que podria ser buena opcion la siguiente:
''' # Realizaremos interpolacion para las fechas nulas, usando el id de creacion del registro,
# para lo cual usaremos las fechas adyacentes para reemplazar los valores NaN
# Debemos garantizar que solo afectaremos los 87 registros nulos y no otros registros, 
# Paso 1: Copiamos la data original para compararla luego con la data resultante, la diferencia debe dar 87 registros transformados.
estado_original = df['release_date'].copy()
# Paso 2: Rellenamos valores faltantes hacia adelante(fecha posterior) o hacia atras (fecha anterior) 
df['release_date'] = df.groupby('id')['release_date'].ffill().bfill()
# Paso 3: Comparar cuántos registros cambiaron para estar seguros de no haber modificado las fechas originales.
registros_cambiados = (estado_original != df['release_date']).sum()
print(f"Registros cambiados: {registros_cambiados}") '''

' # Realizaremos interpolacion para las fechas nulas, usando el id de creacion del registro,\n# para lo cual usaremos las fechas adyacentes para reemplazar los valores NaN\n# Debemos garantizar que solo afectaremos los 87 registros nulos y no otros registros, \n# Paso 1: Copiamos la data original para compararla luego con la data resultante, la diferencia debe dar 87 registros transformados.\nestado_original = df[\'release_date\'].copy()\n# Paso 2: Rellenamos valores faltantes hacia adelante(fecha posterior) o hacia atras (fecha anterior) \ndf[\'release_date\'] = df.groupby(\'id\')[\'release_date\'].ffill().bfill()\n# Paso 3: Comparar cuántos registros cambiaron para estar seguros de no haber modificado las fechas originales.\nregistros_cambiados = (estado_original != df[\'release_date\']).sum()\nprint(f"Registros cambiados: {registros_cambiados}") '

In [229]:
df['original_language'].value_counts(dropna=False).sort_values(ascending=False)
df['original_language'].isnull().sum() ## Contamos los nulos
# Verificamos y encontramos para la columna 'original_language' 11 valores en nulo.
# El valor de los no nulos es cadena sin algun tipo de formato, por lo tanto rellenamos los nulos con la opcion para el modelo "sin preferencias".
df['original_language'] = df['original_language'].fillna('desconocido')

In [230]:
#df['vote_average'].value_counts(dropna=False).sort_values(ascending=False)
#print(df['vote_average'].isnull().sum()) ## Contamos los nulos
# Rellenamos con 0.0 como valor indicativo de no calificacion.
df['vote_average'] = df['vote_average'].fillna(0.0)

In [231]:
# No se evidencias Outlaier
df['vote_count'].value_counts(dropna=False).sort_values(ascending=False)
df['vote_count'].isnull().sum() ## Contamos los nulos
# Rellenamos con 0.0 como valor indicativo de no calificacion.
df['vote_count'] = df['vote_count'].fillna(0.0)

In [232]:
df['title'].isnull().sum()  # contamos los nulos
# Eliminar filas donde la columna 'fecha' sea nula (NaN)
# Encontramos 87 valores nulos en esta columna.
df = df.dropna(subset=['title'])

In [233]:
# Validamos de nuevo los nulos en la data y no se encuentran mas.
df.isnull().sum().sort_values(ascending=False)

belongs_to_collection    0
budget                   0
genres                   0
id                       0
original_language        0
overview                 0
popularity               0
production_companies     0
production_countries     0
release_date             0
revenue                  0
runtime                  0
spoken_languages         0
status                   0
tagline                  0
title                    0
vote_average             0
vote_count               0
dtype: int64

Ahora que no tenemos nulos podemos, desanidar las columnas que contienen diccionarios o listas de diccionarios, sin embargo antes de hacerlo debemos tratar sus campos y valores.

In [234]:
# Una vez corroborados que las columnas a desanidaar este con la caracteristicas de lista de diccionarios,
# Procedemos a crear una sola funcion para todas ellas, pues el patron en comun el la clave 'name' en sus diccionarios
def extraer_valor(cadena):
    if not cadena:  # Verifica si la cadena es None o vacía
        return ""
    # Convierte la cadena en una lista de diccionarios
    lista_diccionarios = ast.literal_eval(cadena)
    # Extrae los nombres y los une en un solo string
    generos = [diccionario['name'] for diccionario in lista_diccionarios if 'name' in diccionario]
    # Une los nombres en un string sin comillas ni llaves
    return ', '.join(generos)  # Une los nombres en un string

#Asignamos a belongs_to_collection [] llaves para que el diccionario quede en una lista.
df['belongs_to_collection'] =  '[' + df['belongs_to_collection'] + ']'
# Aplica la función y guarda el resultado en la columna 'genero'
df['coleccion'] = df['belongs_to_collection'].apply(extraer_valor)
df['genero'] = df['genres'].apply(extraer_valor)
df['compañia_productora'] = df['production_companies'].apply(extraer_valor)
df['ciudad_produccion'] = df['production_countries'].apply(extraer_valor)
df['idiomas_pelicula'] = df['spoken_languages'].apply(extraer_valor)

In [235]:
''' Activando cada linea por separado visualizamos la desanidacion y comparativa
    con el campo original, nos sive para validar si la funcion unica sirve para todos los casos '''
#df[['coleccion','belongs_to_collection']]
#df[['genero','genres']]
#df[['compañia_productora','production_companies']]
#df[['ciudad_produccion','production_countries']]
#df[['idiomas_pelicula','spoken_languages']]

' Activando cada linea por separado visualizamos la desanidacion y comparativa\n    con el campo original, nos sive para validar si la funcion unica sirve para todos los casos '

In [236]:
# Validados los campos desanidados procedemos a eliminar los que no necesitamos
df = df.drop(columns=['belongs_to_collection', 'genres', 'production_companies','production_countries','spoken_languages'])

Convertimos algunos campos a sus tipos de datos correspondientes

In [237]:
# Convertimos el campo a numero.
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['id'] = df['id'].astype(int)
df['budget'] = df['budget'].astype(int)
df['popularity'] = df['popularity'].astype(float)


In [238]:
'''Campo requerido por el proyecto'''
# Se pide crear la columna 'return' calculando el retorno de inversión, para lo cual los dos campos necesarios,
# para esta calculo deben ser numericos, por tanto validamos que budget y revenue solo contengan valores numericos, 
# de lo contrarios los convertiremos en NaNcontiene algunos registros que son cadenas de texto, procedemos a procesarlos.
# Convertir 'revenue' a numérico, forzando a NaN los valores no convertibles
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
# Borrar directamente los datos nulos en la columna 'revenue'
df.dropna(subset=['revenue'], inplace=True)
df['revenue'] = df['revenue'].astype(int)
# De igual forma para el siguiente campo.
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df.dropna(subset=['budget'], inplace=True)
df['budget'] = df['budget'].astype(int)

# Crear la columna 'return' calculando el retorno de inversión
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] and row['budget'] > 0 else 0, axis=1)

In [239]:
'''Campo requerido por el proyecto'''
# Convertir la columna a tipo datetime
df['release_year'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

In [240]:
# Convertir la columna a tipo datetime
#df['release_date'] = pd.to_datetime(df['release_date'])
# Dar formato a la columna de fechas
#df['formatted_date'] = df['release_date'].dt.strftime('%Y-%d-%m')
#df['formatted_date'] = pd.to_datetime(df['formatted_date'], format='%Y-%d-%m')

# Convertir la columna a tipo datetime
# df['release_year'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

In [241]:
# Extraer el año y el mes en columnas separadas
#df['year'] = df['release_date'].dt.year
#df['month'] = df['release_date'].dt.month

In [242]:
filtered_df = df[(df['release_date'] >= '2015-01-01')] # & (df['original_language'] == 'en')]

Antes de proceder a la carga de los datos en los nuevos archivos CSV, procedemos a validar por ultima vez los tipos de datos y los nulos

In [243]:
# Por ultimo y antes de guardar los datos transformados, verificamos los tipos de datos definitivos
df.dtypes

budget                          int64
id                              int64
original_language              object
overview                       object
popularity                    float64
release_date                   object
revenue                         int64
runtime                       float64
status                         object
tagline                        object
title                          object
vote_average                  float64
vote_count                    float64
coleccion                      object
genero                         object
compañia_productora            object
ciudad_produccion              object
idiomas_pelicula               object
return                        float64
release_year           datetime64[ns]
dtype: object

In [244]:
# Por ultimo y antes de guardar los datos transformados, verificamos los si existen otros nulos
df.isnull().sum().sort_values(ascending=False)

budget                 0
id                     0
original_language      0
overview               0
popularity             0
release_date           0
revenue                0
runtime                0
status                 0
tagline                0
title                  0
vote_average           0
vote_count             0
coleccion              0
genero                 0
compañia_productora    0
ciudad_produccion      0
idiomas_pelicula       0
return                 0
release_year           0
dtype: int64

In [245]:
''' Activando cada linea por separado visualizamos los registros resultantes.'''
#print(df['caracteristicas_pelicula'].iloc[0])
#df[['overview','popularity','tagline','vote_average','vote_count','genero','ciudad_produccion','idiomas_pelicula']]

' Activando cada linea por separado visualizamos los registros resultantes.'

In [246]:
''' De la siguiente manera podemos validar uno a uno los campos para identificar cuantos vacios
    tiene y determinar si se eliminan o no'''

#vacios = (df['tagline'] == ' ').sum()
#vacios

' De la siguiente manera podemos validar uno a uno los campos para identificar cuantos vacios\n    tiene y determinar si se eliminan o no'

Realizamos el cargue de los datos en sus archivos CSV

In [247]:
# Cortar el dataset
filtered_df = df[(df['release_date'] >= '2015-01-01')] # & (df['original_language'] == 'en')]


# Debido a la restriccion de 512MB de render, debemos recortar la data, tanto de registros, como campos no necesarios.
df_final_movies = filtered_df.head(5000)
df_fl = df_final_movies[['popularity','release_date','release_year','title','vote_average','vote_count']]
# Guardamos los registros en CSV
df_fl.to_csv('Dataset/Procesados/data_movies.csv', index=False)

In [248]:
#Unificamos algunas columnas para la funcion API con modelo de recomendacion basado en la similitud del coseno.
df['caracteristicas_pelicula'] = (
    df['overview'] + " " +
    df['popularity'].astype(str) + " " +
    df['vote_average'].astype(str) + " " +
    df['vote_count'].astype(str) + " " +
    df['genero'] + " " +
    df['idiomas_pelicula'])

# Crear una nueva columna 'recomienda' (si es tu objetivo) con los mismos datos
df_recomienda = df[['title','caracteristicas_pelicula']]
# Eliminamos los registros duplicados en su titulo de pelicula
df_recomienda.drop_duplicates(subset='title', keep='first', inplace=True)

# Debido a la restriccion de 512MB de render, debemos recortar la data a pocos registros.
df_final_recomienda = df_recomienda.head(4000)
# Guardamos los registros en CSV
df_final_recomienda.to_csv('Dataset/Procesados/data_recomienda.csv', index=False)

C:\Users\Dalgy\AppData\Local\Temp\ipykernel_19408\1519794509.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recomienda.drop_duplicates(subset='title', keep='first', inplace=True)


Convertir a formato parquet

In [249]:
# filtered_df.to_parquet('data.parquet', index=False)

# print("DataFrame guardado en formato Parquet sin índice.")